# Extraire la legende depuis la description
## tentative n°1

https://regex101.com/

https://docs.python.org/2/library/re.html#module-contents

In [276]:
import pickle
import regex   # permet overlapping matching

In [277]:
def print_legend( legend ):
    sortfun = lambda x: ( x['number'] , len(x['position']), x['label'][::-1] )
    legendsorted = sorted( legend, key=sortfun  )

    current_number = legendsorted[0]['number']
    for match in legendsorted:
        if current_number != match['number']:
            print('     --')
            current_number = match['number']

        n_merged = len(match['position'])
        n_merged_str = '  ' if n_merged==1 else '(%i)'%n_merged
        print( '{label:>42} {number:>4}  {n}  {context:>35}'.format( **match, n=n_merged_str) )

In [278]:
# load pre-processed data
data = pickle.load( open( "../web/data/patent_infos.pickle", "rb" ) )
len(data)

379

In [279]:
# select a patent

k = 70 #44 #41
patent = list(data.values())[k]
description = patent['description']

print( patent['patent_number'] )
print( 'https://www.google.com/patents/%s'%patent['patent_number'] )

print('nombre de mots :', len(description.split(' ')))

US20160286931
https://www.google.com/patents/US20160286931
nombre de mots : 4215


In [280]:
# Travail sur le texte entier :

description = description.replace('\n', '')
description = regex.sub('\s+',' ', description) # remove multiple space
description = description.lower()

# patent number XX
patent_number_pattern = r'(\d{1, 4}[,-])?\d{3, 8}[,-]\d{3, 8}'
description = regex.sub(patent_number_pattern, '', description  )

# remove non-AASII caracters
description = regex.sub(r'[^\x00-\x7F]+', '', description)

In [281]:
# Find values
units = ['inch', 'volt', 'm', 'mm']
find_sci_values =  r" (\d{1,4}([\.,]\d{1,4})? (%s)\W)" % '|'.join(units)

print( regex.findall(find_sci_values, description) )
description = regex.sub(find_sci_values, '<value>', description  )

[]


In [282]:
# Find values
units = ['inch', 'volt', 'm', 'mm']
find_sci_values =  r" (\d{1,4}([\.,]\d{1,4})? (%s)\W)" % '|'.join(units)

print( regex.findall(find_sci_values, description) )
description = regex.sub(find_sci_values, '<value>', description  )

[]


In [283]:
# Find floats
find_sci_floats =  r" (\d{1,4}([\.,]\d{1,4})\W)" 

print( regex.findall(find_sci_floats, description) )
description = regex.sub(find_sci_floats, '<value>', description  )

[('3,855,', ',855'), ('3,997,', ',997'), ('3,169,', ',169'), ('4,602,', ',602'), ('4,731,', ',731'), ('5,195,', ',195')]


In [284]:
# " between 0.4 to 2.5 " 
betweenpattern = r"(between [\d\.,]{1,4} (to|and) [\d\.,]{1,4}\W)"

print( regex.findall(betweenpattern, description) )
description = regex.sub(betweenpattern, '<between>', description  )

[]


In [285]:
description

'related applications [0001] this application is a continuation application that claims priority to a non-provisional application ser. no. 14/474,138 filed on aug. 31, 2014 and incorporated herewith by reference in its entirety. field of the invention [0002] the present invention relates to nail clipping and cutting devices and more particularly to nail clippings receptacles having enhancements that improve the receptacle&#39;s functionality. background of the invention [0003] people of all ages trim and clip nails on daily bases. nail clippers became a necessity as people cannot get by especially during travel. when a person trims his nails, the nail clippings have a tendency to spring away from the nail clipper in any number of unpredictably different directions. however, few people want to put up with the hassle of and spend time for searching for stray nail clippings. for this reason there is a strongly perceived need for nail clippings receptacles for use with nail clippers. many 

In [286]:
# trouve tout les chiffres, avec un espace devant
#exp =  r"(.{40}) (\d{1,3})"

exp =  r" (?P<n1>\d{1,3})( (and|or) (?P<n2>\d{1,3}))?[^a-z0-9\-]"

# exp =  r'(\d{1,3}((, | and )\d{1,3})*)'   # E(SE)  Element  Separator

allmatches = list( regex.finditer(exp, description) )
print(len(allmatches))

legend = []
for m in allmatches:

    label = description[ max(0, m.start()-40 ): m.start() ]  # to prevent overlapping
    context = description[ max(0, m.start()-20 ): min( len(description), m.end()+8 ) ]  # to debug
    
    item = {'number':int(m.group('n1')), 'label':label, 'position':[m.span('n2')], 'context':context}
    legend.append( item )
    
    if m.group('n2'): # correspond to "23 and 24"
        item = {'number':int(m.group('n2')), 'label':label, 'position':[m.span('n2')], 'context':context}
        legend.append( item )
        
print(len(legend))

179
223


In [287]:
# autre approche
#    ' 45, 46 and 47 '

exp =  r' (\d{1,3}(( ?, ?| and | or )\d{1,3})*)[^a-z0-9\-]'

# explication:
# la forme est E(SE) avec Element  Separator
# et le séparateur est une virgule ou and ou or 

splitpat = regex.compile(r'[^\d]+')

allmatches = list( regex.finditer(exp, description) )

legend = []
for m in allmatches:
    capture = m.group(1)
    numbers = splitpat.split( capture )
    start = m.start()
    
    label = description[ max(0, m.start()-40 ): m.start() ]  # to prevent overlapping
    context = description[ max(0, m.start()-10 ): min( len(description), m.end()+10 ) ]  # to debug
    
    for n in numbers:
        item = {'number':int(n), 'label':label, 'position':[m.span(1)], 'context':context}
        legend.append( item )
        
print(len(legend))

223


In [288]:
def disqualify( legend, pattern, n=999 ):
    for i, row in enumerate( legend ):
        if pattern.search( row['label'] ):
            legend[ i ]['number'] = n

# Disqualifie si point final :  & tiret   ... mais pas une virgule car 45, 46 and 47 -> on va le faire avant ...
# Fig. 1  ;  No. 511  
# en fait si ce n'est pas un Word caracter... que l'on restreint à l'alphabet, pour éviter les symboles
pattern = regex.compile( r'[^a-z]$', regex.I )

disqualify( legend, pattern, n=999 )

        
# si c'est un mois, ou un autre mots clés :
keywords = ['january', 'february', 'march', 'april', 'may', 'june',\
          'july', 'august', 'september', 'october', 'november', 'december',\
           'figure', 'figures', 'claim', 'claims', 'at', 'and',\
            'numeral', 'embodiment', 'invention']

keywords.extend( ['the', 'a', 'an', 'these', 'their', 'when', 'with', 'by', 'this', \
               'have', 'having', 'has', 'is', 'are', 'over', 'its', 'of said', 'and', 'as',\
              'of', 'in', 'to', 'but', 'another', 'through', 'on'] )

pattern = regex.compile( r'\W(%s)$' % '|'.join(keywords), regex.I )

disqualify( legend, pattern, n=998 )
        
    
print_legend( legend )

  lipping device. the nail clipping device   10                 ing device 10 includes a
  s. 1 through 4, 7, and 8 , the apparatus   10                  apparatus 10 of the pre
   alternative embodiment of the apparatus   10                  apparatus 10 of the pre
   alternative embodiment of the apparatus   10                  apparatus 10 of the pre
     --
  ipping device 10 includes a nail clipper   12                 il clipper 12 and a hous
  e the nail clipper 12 . the nail clipper   12                 il clipper 12 includes a
  42 adaptable to receive the nail clipper   12                 il clipper 12 . [0034] t
  14 adaptable to receive the nail clipper   12                 il clipper 12 . the nail
  nd preventing escape of the nail clipper   12                 il clipper 12 from the h
  ral axis a for securing the nail clipper   12                 il clipper 12 within the
     --
  includes a nail clipper 12 and a housing   14                  a housing 14 adaptable 
  pe 

  inal ends 130 to another of terminal end  132                rminal end 132 . the top 
   therein. the other of the terminal ends  132                minal ends 132 is further
  l axis a extending between terminal ends  132        minal ends 130 and 132 wherein th
     --
  18 , and a top wall 140 . the side walls  134        side walls 134 and 136 may presen
   ends 130 and 132 wherein the side walls  134        side walls 134 and 136 diverging 
  nd 132 . the housing includes side walls  134        side walls 134 and 136 , a bottom
     --
  18 , and a top wall 140 . the side walls  136        side walls 134 and 136 may presen
   ends 130 and 132 wherein the side walls  136        side walls 134 and 136 diverging 
  nd 132 . the housing includes side walls  136        side walls 134 and 136 , a bottom
     --
  136 , a bottom wall 318 , and a top wall  140                a top wall 140 . the side
  other of terminal end 132 . the top wall  140                e top wall 140 includes

## Coupe sur certain mot clé (the, a, an, ...)

In [289]:
def cut_label( legend, pattern ):
    for i, row in enumerate( legend ):
        label = row['label']

        match = pattern.search(label)

        if match:
            newlabel = match.group(2)
            legend[i]['label'] = newlabel

In [290]:
small_words = ['the', 'a', 'an', 'these', 'their', 'when', 'with', 'by', 'this', 'that', \
               'have', 'having', 'has', 'is', 'are', 'should', 'over', 'its', 'of said', 'and', 'as',\
              'of', 'in', 'to', 'at',  'but', 'another', 'through', 'on', 'same', 'from',\
              'include', 'includes', 'beyond', 'between']

pattern = regex.compile( r'^.* (%s) (.+)$' % '|'.join(small_words), regex.I )

cut_label( legend, pattern )

In [291]:
# cut at point and comma :
pattern = regex.compile( '^.*([\.,;] )(.+)$' )

cut_label( legend, pattern )

print_legend( legend )

                      nail clipping device   10                 ing device 10 includes a
                                 apparatus   10                  apparatus 10 of the pre
                                 apparatus   10                  apparatus 10 of the pre
                                 apparatus   10                  apparatus 10 of the pre
     --
                              nail clipper   12                 il clipper 12 and a hous
                              nail clipper   12                 il clipper 12 . the nail
                              nail clipper   12                 il clipper 12 includes a
                              nail clipper   12                 il clipper 12 . [0034] t
                              nail clipper   12                 il clipper 12 within the
                              nail clipper   12                 il clipper 12 from the h
     --
                                   housing   14                  a housing 14 adaptable 
     

In [292]:
def get_positions( label, candidats ):
    positions = []
    for item in candidats:
        if item['label']==label:
            positions.extend( item['position'] )
    return positions

def get_context( label, candidats ):
    items = [ item for item in candidats if item['label']==label ]
    
    return items[0]['context']

def merge( legend ):
    # merge identical label
    
    numbers_unique = { item['number'] for item in legend  } # set

    new_legend = []
    for n in numbers_unique:

        candidats = [ line for line in legend if line['number']==n ]

        labels_unique = { item['label'] for item in candidats  }

        new_candidats = [ { 'number':n, 'label':label_u,\
                           'position':get_positions(label_u, candidats), 'context':get_context(label_u, candidats) }
                          for label_u in labels_unique ]
        
        new_legend.extend( new_candidats )

    return new_legend

In [293]:
print_legend(  merge( legend ) )

                      nail clipping device   10                 ing device 10 includes a
                                 apparatus   10  (3)              apparatus 10 of the pre
     --
                              nail clipper   12  (6)             il clipper 12 and a hous
     --
                                   housing   14  (8)              a housing 14 adaptable 
     --
                           members or arms   16          rs or arms 16 and 18 each prese
                                   members   16          he members 16 and 18 and the cu
                                      arms   16  (2)      h the arms 16 and 18 as known t
     --
                           members or arms   18          rs or arms 16 and 18 each prese
                                   members   18          he members 16 and 18 and the cu
                                      arms   18  (2)      h the arms 16 and 18 as known t
     --
          18 each presenting cutting edges   20          ting edg

In [294]:
mergedlegend = merge( legend )

In [295]:
mergedlegend[0]

{'context': 'minal ends 130 and 132 wherein th',
 'label': 'terminal ends',
 'number': 130,
 'position': [(15203, 15214), (15290, 15293), (15806, 15809)]}

In [296]:
i = 48
candidates = [ c for c in mergedlegend if c['number'] == i ]

vainqueur = 'notyet'

if len(candidats) == 1:
    vainqueur = candidats[0]
else:
    nbreOccurences = [ len( c['position'] ) for c in candidats ]
    
    if isMaxUnique( nbreOccurences ):
        argmax = nbreOccurences.index( max(nbreOccurences) )
        vainqueur = candidats[argmax]
    else:
        print('on recommence')
    
print( vainqueur )

{'number': 48, 'context': ' telescope 48 is secured', 'position': [(14491, 14493), (14777, 14779), (15683, 15685), (15981, 15983), (17510, 17512), (17859, 17861)], 'label': 'telescope', 'reversed': ['telescope']}


In [297]:
def isMaxUnique( intlist ):
    return True if intlist.count( max( intlist ) ) == 1 else False

In [298]:
i = 17
candidates = [ c for c in mergedlegend if c['number'] == i ]

for c in candidates:
    c['reversed'] = c['label'].split(' ')[::-1]
    c['weight'] = len( c['position'] )

In [299]:
for c in candidates:
    print( ', '.join(c['reversed']) )

In [300]:
def incrementscore( dico, key, value ):
    if key in dico:
        dico[key] += value
    else:
        dico[key] = value
        
def defaultappend( dico, key, value ):
    if key in dico:
        dico[key].append( value )
    else:
        dico[key] = [value]

In [301]:
def whoAreTheWinners( scoreboard ):
    scoreMax = max( scoreboard.values() )
    winners = [ sb[0] for sb in scoreboard.items() if sb[1] == scoreMax ]
    return winners

In [302]:
remainingcandidates = candidates
previouschoice=[]
for i in range(20):
    current_indice = len(previouschoice)
    
    scoreboard = {}
    nextword2candidates ={}
    for c in remainingcandidates:
        if current_indice < len( c['reversed'] ):
            next_word = c['reversed'][ current_indice ]
        else:
            next_word = ''
            
        defaultappend( nextword2candidates, next_word, c )
        incrementscore( scoreboard, next_word, c['weight'] )
        
        #print(next_word, c['weight'])
        
    print( 'scoreboard: ', scoreboard )
    winners = whoAreTheWinners( scoreboard )
    print( 'winners:', winners )
    if len(winners) == 1 and winners[0]:
        choice = winners[0]
        remainingcandidates = nextword2candidates[choice]
        previouschoice.append( choice )
        print(' ')
        
    else: # exaequo ou fin -> stop
        label = ' '.join( previouschoice[::-1] )
        print( label )
        break
        
    

scoreboard:  {}


ValueError: max() arg is an empty sequence

In [ ]:
for c in candidates:
    print( c['weight'], ', '.join(c['reversed']) )

In [ ]:
choose( candidates, previouschoice=['elements'] )

### Comment selectionner le meilleurs candidats ?

 * si unique:  ok
       et si unique et len()==40... ? c.a.d. pas coupé
 * si max unique: ok
 * sinon : selectionne les qualifiés, puis...
 * exaequo...  construit un arbre de comptage ?




### To look
* comment ne pas modifier le texte, pour pouvoir l'annoter ?
* paranthese: between the elongated plates ( 1 , 2 ).
* line 5 - 5
*      march   15                 march 15,   
en debut de la description ...
* generally... adverbes ?
* " respectivement A et B " ... respective
* letter sur les vieux brevets
* merge keywords and small_words ??
* pluriel

* sort of recursive... identifier un permet d'en déduire un autre...

## other things
* Find values like "110 volts"... 